This notebook is part of https://github.com/AudioSceneDescriptionFormat/splines, see also https://splines.readthedocs.io/.

# Derivation of Non-Uniform Catmull--Rom Splines

Multi-stage algorithm developed by
<cite data-cite="barry1988recursive">Barry and Goldman (1988)</cite>,
according to
<cite data-cite="yuksel2011parameterization">Yuksel et al. (2011)</cite>, figure 3,
which looks somewhat like this (but we shifted the indices by $-1$):

\begin{equation*}
\begin{array}{ccccccccccccc}
&&&&&&
\boldsymbol{x}_{0,1}
&&&&&&
\\
&&&&&
\frac{t_1 - t}{t_1 - t_0}
&&
\frac{t - t_0}{t_1 - t_0}
&&&&&
\\
&&&& \boldsymbol{p}_{-1,0,1} &&&& \boldsymbol{p}_{0,1,2} &&&&\\
\\
&&
& \frac{t_1 - t}{t_1 - t_{-1}} && \frac{t - t_{-1}}{t_1 - t_{-1}} &
& \frac{t_2 - t}{t_2 - t_0} && \frac{t - t_0}{t_2 - t_0} &
&&
\\
&& \boldsymbol{p}_{-1,0} &&&& \boldsymbol{p}_{0,1} &&&& \boldsymbol{p}_{1,2} &&
\\
& \frac{t_0 - t}{t_0 - t_{-1}} && \frac{t - t_{-1}}{t_0 - t_{-1}} &
& \frac{t_1 - t}{t_1 - t_0} && \frac{t - t_0}{t_1 - t_0} &
& \frac{t_2 - t}{t_2 - t_1} && \frac{t - t_1}{t_2 - t_1} &
\\
\boldsymbol{x}_{-1} &&&& \boldsymbol{x}_0 &&&& \boldsymbol{x}_1 &&&& \boldsymbol{x}_2
\end{array}
\end{equation*}

Here we are considering the spline segment
$\boldsymbol{x}_{0,1}(t)$
from
$\boldsymbol{x}_0$ to
$\boldsymbol{x}_1$ which corresponds to
a range of the parameter $t$
from $t_0$ to $t_1$ (represented at the tip of the triangle).
To calculate the values in this segment,
we also need to know the preceding control point $\boldsymbol{x}_{-1}$
(at the bottom left)
and the following control point $\boldsymbol{x}_2$
(at the bottom right).
But not only their positions are relevant,
we also need the corresponding parameter values
$t_{-1}$ and $t_2$, respectively.

## Preparations

Let's import [SymPy](https://www.sympy.org/)
and define the symbols we need:

In [ ]:
import sympy as sp
sp.init_printing()

In [ ]:
x_1, x0, x1, x2 = sp.symbols('xbm_-1 xbm:3')

In [ ]:
t, t_1, t0, t1, t2 = sp.symbols('t t_-1 t:3')

We also use some custom SymPy tools from [utility.py](utility.py):

In [ ]:
from utility import NamedExpression, NamedMatrix

The triangular figure above looks more complicated than it really is.
It's just a bunch of linear *inter*polations and *extra*polations.
Since we'll need several of those,
let's define a helper function:

In [ ]:
def lerp(xs, ts):
    """Linear interpolation.
    
    Between the two points given by *xs* in the time span given by *ts*.
    
    """
    x_begin, x_end = xs
    t_begin, t_end = ts
    return (x_begin * (t_end - t) + x_end * (t - t_begin)) / (t_end - t_begin)

Let's go through the figure above, piece by piece.

## First Stage

In the center of the bottom row,
there is a straightforward linear interpolation
from $\boldsymbol{x}_0$ to $\boldsymbol{x}_1$
within the interval from $t_0$ to $t_1$.

In [ ]:
p01 = NamedExpression('pbm_0,1', lerp((x0, x1), (t0, t1)))
p01

Obviously, this starts at:

In [ ]:
p01.evaluated_at(t, t0)

... and ends at:

In [ ]:
p01.evaluated_at(t, t1)

The bottom left of the triangle looks very similar,
with a linear interpolation
from $\boldsymbol{x}_{-1}$ to $\boldsymbol{x}_0$
within the interval from $t_{-1}$ to $t_0$.

In [ ]:
p_10 = NamedExpression('pbm_-1,0', lerp((x_1, x0), (t_1, t0)))
p_10

However, that's not the parameter range we are interested in.
We are interested in the range from $t_0$ to $t_1$.
Therefore, this is not actually an *inter*polation between
$\boldsymbol{x}_{-1}$ and $\boldsymbol{x}_0$,
but rather a linear *extra*polation starting at $\boldsymbol{x}_0$:

In [ ]:
p_10.evaluated_at(t, t0)

... and ending at some extrapolated point beyond $\boldsymbol{x}_0$:

In [ ]:
p_10.evaluated_at(t, t1)

Similarly, at the bottom right of the triangle
there isn't a linear *inter*polation
from $\boldsymbol{x}_1$ to $\boldsymbol{x}_2$,
but rather a linear *extra*polation that just reaches
$\boldsymbol{x}_1$ at the end of the parameter interval
(i.e. at $t=t_1$).

In [ ]:
p12 = NamedExpression('pbm_1,2', lerp((x1, x2), (t1, t2)))
p12

In [ ]:
p12.evaluated_at(t, t0)

In [ ]:
p12.evaluated_at(t, t1)

## Second Stage

The second stage of the algorithm
involves linear interpolations of the results of the previous stage.

In [ ]:
p_101 = NamedExpression('pbm_-1,0,1', lerp((p_10.name, p01.name), (t_1, t1)))
p_101

In [ ]:
p012 = NamedExpression('pbm_0,1,2', lerp((p01.name, p12.name), (t0, t2)))
p012

Those interpolations are defined over a parameter range
from $t_{-1}$ to $t_1$ and
from $t_0$ to $t_2$, respectively.
In each case, we are only interested in a sub-range,
namely from $t_0$ to $t_1$.

These are the start and end points at $t_0$ and $t_1$:

In [ ]:
p_101.evaluated_at(t, t0, symbols=[p_10, p01])

In [ ]:
p_101.evaluated_at(t, t1, symbols=[p_10, p01])

In [ ]:
p012.evaluated_at(t, t0, symbols=[p01, p12])

In [ ]:
p012.evaluated_at(t, t1, symbols=[p01, p12])

## Third Stage

The last step is quite simple:

In [ ]:
x01 = NamedExpression('xbm_0,1', lerp((p_101.name, p012.name), (t0, t1)))
x01

This time, the interpolation interval is exactly the one we care about.

To get the final result, we just have to combine all the above expressions:

In [ ]:
x01 = x01.subs_symbols(p_101, p012, p_10, p01, p12).simplify()
x01

We can make this marginally shorter
if we rewrite the parameter values as
$\Delta_i = t_{i+1} - t_i$:

In [ ]:
deltas = [
    (t_1, -sp.Symbol('Delta_-1')),
    (t0, 0),
    (t1, sp.Symbol('Delta0')),
    (t2, sp.Symbol('Delta0') + sp.Symbol('Delta1'))
]

In [ ]:
x01.expr.subs(deltas)

## Characteristic Matrix

We already have the correct result,
but if we want to derive our "characteristic matrix",
we have to re-scale this a bit.
The parameter is supposed to go from $0$ to $1$
instead of from $t_0$ to $t_1$:

In [ ]:
x01_normalized = x01.expr.subs(t, t * (t1 - t0) + t0).subs(deltas)

In [ ]:
M_CR = NamedMatrix(
    r'{M_\text{CR}}',
    sp.Matrix([[c.expand().coeff(x).factor() for x in (x_1, x0, x1, x2)]
               for c in x01_normalized.as_poly(t).all_coeffs()]))

In [ ]:
M_CR

And just to make sure that is consistent with the result
from [uniform Catmull--Rom splines](catmull-rom-uniform.ipynb),
let's set all $\Delta_i$ to $1$:

In [ ]:
uniform = [
    (sp.Symbol('Delta_-1'), 1),
    (sp.Symbol('Delta0') , 1),
    (sp.Symbol('Delta1') , 1),
]

In [ ]:
M_CR_uniform = NamedMatrix(
    r'{M_\text{CR,uniform}}',
    M_CR.expr.subs(uniform))

In [ ]:
M_CR_uniform.pull_out(sp.S.Half)

## Begin/End Tangents

To get the tangents at $t_0$ and $t_1$,
we just have to differentiate:

In [ ]:
x_dot = NamedExpression('xdotbm', x01.expr.diff(t))

In [ ]:
start_tangent = x_dot.evaluated_at(t, t0)
start_tangent.subs(deltas).factor()

In [ ]:
end_tangent = x_dot.evaluated_at(t, t1)
end_tangent.subs(deltas).factor()

in general (just adding $i$ to all indices):

\begin{equation}
\boldsymbol{\dot{x}}_i =
\frac{
(t_{i+1} - t_i)^2 (\boldsymbol{x}_i - \boldsymbol{x}_{i-1}) +
(t_i - t_{i-1})^2 (\boldsymbol{x}_{i+1} - \boldsymbol{x}_i)
}{
(t_{i+1} - t_i)(t_i - t_{i-1})(t_{i+1} - t_{i-1})
}
\end{equation}

You might encounter another way to write the equation for $\boldsymbol{\dot{x}}_0$
(e.g. at https://stackoverflow.com/a/23980479/):

In [ ]:
(x0 - x_1) / (t0 - t_1) - (x1 - x_1) / (t1 - t_1) + (x1 - x0) / (t1 - t0)

... but this is equivalent to the equation shown above:

In [ ]:
sp.simplify(_ - start_tangent.expr)

Yet another way to skin this cat -- sometimes referred to as Bessel--Overhauser -- is to define the velocity of the left and right chords:

In [ ]:
v_left = (x0 - x_1) / (t0 - t_1)
v_right = (x1 - x0) / (t1 - t0)

... and then combine them in this way:

In [ ]:
((t1 - t0) * v_left + (t0 - t_1) * v_right) / (t1 - t_1)

Again, that's the same as we had above:

In [ ]:
sp.simplify(_ - start_tangent.expr)

## Animation

The linear interpolations (and *extra*polations) of this algorithm
can be shown graphically.

By means of the file [barry_goldman.py](barry_goldman.py),
we can generate animations of the algorithm:

In [ ]:
from barry_goldman import animation

In [ ]:
from IPython.display import HTML

In [ ]:
points = [
    (0, 0),
    (0.5, 1),
    (6, 1),
    (6.5, 0),
]

In [ ]:
times = [
    0,
    1,
    5,
    9,
]

In [ ]:
ani = animation(points, times)

In [ ]:
HTML(ani.to_jshtml(default_mode='reflect'))